In [2]:
!pip install -q pyworld

     |████████████████████████████████| 212 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [77]:
num = 0

In [78]:
import pyworld as pw
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

In [79]:
path = '/content/drive/MyDrive/nus-smc-corpus_48/ADIZ'

In [80]:
Adiz_read, fs = librosa.load(path +'/read/01.wav', sr = None)
f0, sp, ap = pw.wav2world(Adiz_read.astype(float), fs)

In [81]:
Adiz_sing, fs1 = librosa.load(path + '/sing/01.wav', sr = None)
f01, sp1, ap1 = pw.wav2world(Adiz_sing.astype(float), fs1)

In [82]:
read_df = pd.read_csv(path + "/read/01.txt", delimiter=" ", header=None, names=['start', 'end', 'phoneme'])
# samf_path = '/content/drive/MyDrive/nus-smc-corpus_48/SAMF/read'
# read_df = pd.read_csv(samf_path + "/01.txt", delimiter=" ", header=None, names=['start', 'end', 'phoneme'])
# Samf_read, fs = librosa.load(samf_path +'/01.wav', sr = None)

sing_df = pd.read_csv(path + "/sing/01.txt", delimiter=" ", header=None, names=['start', 'end', 'phoneme'])

In [25]:
# t_read = librosa.get_duration(y=Adiz_read, sr=fs)
# t_sing = librosa.get_duration(y=Adiz_sing, sr=fs1)
# new_Adiz_read = librosa.effects.time_stretch(Adiz_read, t_read/t_sing)
# f0, spx, apx = pw.wav2world(new_Adiz_read.astype(float), fs)
# y = pw.synthesize(f01, spx, apx, fs)
# sf.write("project.wav", y, fs)

In [84]:
prev = 0
read = []
for i in range(len(read_df)):
  dur_read = read_df['end'][i] - read_df['start'][i]
  dur_sing = sing_df['end'][i] - sing_df['start'][i]
  cur = int(dur_read*fs)+prev

  y = Adiz_read[prev:cur]
  rate = dur_read/dur_sing
  stft = librosa.core.stft(y, n_fft = 512)
  stft_stretch = librosa.core.phase_vocoder(stft, rate)
  len_stretch = int(round(len(y) / rate))
  Adiz_new_read = librosa.core.istft(stft_stretch, dtype=y.dtype, length=len_stretch)

  # Adiz_new_read = librosa.effects.time_stretch(Adiz_read[prev:cur], dur_read/dur_sing)
  if sing_df['phoneme'][i] == 'sil':
    Adiz_read[prev:cur] = 0
  read.append(Adiz_new_read)
  prev = cur

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


In [85]:
read = np.hstack(np.asarray(read))
read_f0, read_sp, read_ap = pw.wav2world(read.astype(float), fs)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [86]:
num += 1
y = pw.synthesize(f01[:len(read_f0)], read_sp, read_ap, fs)
save_path = '/content/drive/MyDrive/MLSP_project/outputs/f0_dur_control_out/'
sf.write(save_path + "project_{}".format(num)+'.wav', y, fs)